###


### Create a dataset using fake library
#### Data set will contain 3 columns id, Date, Transaction

In [46]:
from faker import Faker

import datetime
import pandas as pd
import random



In [47]:
fake = Faker()

In [48]:

def generate_dataset(num_rows):
    dataset = []
    for i in range(num_rows):
        row = [
            i + 1,
            fake.date_between_dates(date_start=datetime.date(2023, 1, 1), date_end=datetime.date.today()),
            f"{random.choice(['+', '-'])}{fake.random_int(min=1, max=999999)}.{fake.random_int(min=0, max=9)}"
        ]
        dataset.append(row)
    return dataset

In [49]:
num_rows = 100
dataset = generate_dataset(num_rows)

In [50]:

columns_lst = ['Id', 'Date', 'Transaction']
df_dataset = pd.DataFrame(dataset, columns=columns_lst)

df_dataset.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           100 non-null    int64 
 1   Date         100 non-null    object
 2   Transaction  100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [51]:
df_dataset.to_csv('stori_ove.csv', index=False)

### Read file

In [52]:
df_dataset['Transaction'] = df_dataset['Transaction'].astype('float64')

total_balance = df_dataset['Transaction'].sum().round(2)

In [53]:
df_dataset['Date'] = pd.to_datetime(df_dataset['Date'])
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Id           100 non-null    int64         
 1   Date         100 non-null    datetime64[ns]
 2   Transaction  100 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 2.5 KB


In [63]:
df_dataset['Month'] = df_dataset['Date'].dt.to_period('M').astype('str')

# df_dataset['Month'] = df_dataset['Month'].astype('str')

df_grouped = df_dataset.groupby('Month').agg( num_transactions=pd.NamedAgg(column='Transaction', aggfunc='count'), avg_credit=pd.NamedAgg(column='Transaction', aggfunc=lambda x: x[x > 0].mean()), avg_debit=pd.NamedAgg(column='Transaction', aggfunc=lambda x: x[x < 0].mean())).reset_index()
                                  
df_grouped['avg_credit']  = df_grouped['avg_credit'].round(2)
df_grouped['avg_debit']  = df_grouped['avg_debit'].round(2)
df_grouped['total_balance']  = total_balance

df_grouped.to_dict(orient='dict')

{'Month': {0: '2023-01',
  1: '2023-02',
  2: '2023-03',
  3: '2023-04',
  4: '2023-05',
  5: '2023-06',
  6: '2023-07',
  7: '2023-08'},
 'num_transactions': {0: 16, 1: 11, 2: 27, 3: 6, 4: 9, 5: 11, 6: 9, 7: 11},
 'avg_credit': {0: 512067.2,
  1: 583267.26,
  2: 533694.26,
  3: 624000.6,
  4: 252623.45,
  5: 562292.38,
  6: 701017.95,
  7: 599784.7},
 'avg_debit': {0: -447522.17,
  1: -653828.3,
  2: -506261.48,
  3: -438965.28,
  4: -557512.73,
  5: -558372.68,
  6: -480433.3,
  7: -548410.07},
 'total_balance': {0: -3491417.7,
  1: -3491417.7,
  2: -3491417.7,
  3: -3491417.7,
  4: -3491417.7,
  5: -3491417.7,
  6: -3491417.7,
  7: -3491417.7}}

In [61]:
def get_balance_dict(df_balance):
    
    result_lst = []
    result_dict = {}
    balance_fl = 0.0
    
    for index, row in df_balance.iterrows():
        row_dict = {
            'Month': row['Month'],
            'num_transactions': row['num_transactions'],
            'avg_credit': row['avg_credit'],
            'avg_debit': row['avg_debit']
        }
        
        result_lst.append(row_dict)
        balance_fl = row['total_balance'] 
        
    result_dict['months'] = result_lst
    result_dict['total_balance'] = balance_fl
    
    return result_dict

In [62]:
print(get_balance_dict(df_grouped))

{'months': [{'Month': '2023-01', 'num_transactions': 16, 'avg_credit': 512067.2, 'avg_debit': -447522.17}, {'Month': '2023-02', 'num_transactions': 11, 'avg_credit': 583267.26, 'avg_debit': -653828.3}, {'Month': '2023-03', 'num_transactions': 27, 'avg_credit': 533694.26, 'avg_debit': -506261.48}, {'Month': '2023-04', 'num_transactions': 6, 'avg_credit': 624000.6, 'avg_debit': -438965.28}, {'Month': '2023-05', 'num_transactions': 9, 'avg_credit': 252623.45, 'avg_debit': -557512.73}, {'Month': '2023-06', 'num_transactions': 11, 'avg_credit': 562292.38, 'avg_debit': -558372.68}, {'Month': '2023-07', 'num_transactions': 9, 'avg_credit': 701017.95, 'avg_debit': -480433.3}, {'Month': '2023-08', 'num_transactions': 11, 'avg_credit': 599784.7, 'avg_debit': -548410.07}], 'total_balance': -3491417.7}


##### Sender teststori25@gmail.com Stori123*